## 과일 사진 데이터 준비하기

In [ ]:
# 캐글에 공개된 과일 데이터 셋 불러오기
# 코랩의 코드 셀에서 '!' 로 시작하면 파이썬 코드가 아닌 리눅스 셀 명령으로 이행함

!wget https://bit.ly/fruits_300_data -O fruits_300.npy

In [ ]:
# k-평균 모델을 훈련하기 위해 (샘플 개수, 너비, 높이) 크기의 3차원 배열을 (샘플 개수, 너비x높이) 크기를 가진 2차원 배열로 변경 (fruits_2d)

import numpy as np

fruits = np.load('fruits_300.npy')
fruits_2d = fruits.reshape(-1, 100*100)

In [ ]:
# K-평균을 구하기 위해 사이킷런 라이브러리의 KMeans 클래스를 불러옴
# 비지도 학습이므로 fit() 함수에서 타깃 데이터를 사용하지 않음

from sklearn.cluster import KMeans

# 클러스터 개수는 3개로 지정
N_cluster = 3

km = KMeans(n_clusters=N_cluster, n_init=10, random_state=42)
km.fit(fruits_2d)

In [ ]:
# 군집화한 label 출력 : 클러스터 개수를 3 개로 지정했으므로 label은 0, 1, 2 중 하나가 됨
# label된 값이 어떤 과일 사진을 주로 모았는지는 알 수 없음. 직접 이미지를 출력해봐야 알 수 있음

print(km.labels_)

In [ ]:
# 각 레이블들이 모은 숫자를 출력해봄

#print(np.unique(km.labels_, return_counts=True))
label, counts = np.unique(km.labels_, return_counts=True)

for i in range(N_cluster) :
  print(f"레이블 {label[i]} 인 이미지 사진 숫자 : {counts[i]}")


In [ ]:
# 각 클러스터가 어떤 이미지를 나타냈는지 그림으로 출력하기 위한 함수 정의

import matplotlib.pyplot as plt

# 적절한 크기로 화면에 출력하기 위해 ratio 인자를 이용용
def draw_fruits(arr, ratio=1):
    n = len(arr)    # n은 샘플 개수입니다
    # 한 줄에 10개씩 이미지를 그립니다. 샘플 개수를 10으로 나누어 전체 행 개수를 계산합니다. 
    rows = int(np.ceil(n/10))
    # 행이 1개 이면 열 개수는 샘플 개수입니다. 그렇지 않으면 10개입니다.
    cols = n if rows < 2 else 10
    fig, axs = plt.subplots(rows, cols, 
                            figsize=(cols*ratio, rows*ratio), squeeze=False)
    for i in range(rows):
        for j in range(cols):
            if i*10 + j < n:    # n 개까지만 그립니다.
                axs[i, j].imshow(arr[i*10 + j], cmap='gray_r')
            axs[i, j].axis('off')
    plt.show()

In [ ]:
# 레이블이 0 인 이미지 출력

draw_fruits(fruits[km.labels_==0], 0.6)

In [ ]:
# 레이블이 1 인 이미지 출력

draw_fruits(fruits[km.labels_==1], 0.6)

In [ ]:
# 레이블이 2 인 이미지 출력

draw_fruits(fruits[km.labels_==2], 0.6)

In [ ]:
# 반복적으로 클러스터 중심을 옮기면서 최적의 클러스터를 찾기 위해 알고리즘이 반복한 횟수

print(km.n_iter_)

### 각 이미지와 클러스터 중심사이의 거리 측정해보기

In [ ]:
# 300 개의 이미지 중 idx번째 이미지 사진에서 클러스터 중심까지 거리 측정

idx = 50

distance = km.transform(fruits_2d[idx:idx+1])
for i in range(N_cluster) :
  print(f"레이블 {i}와의 거리 : {distance[0][i]}")
print(f"따라서 레이블 {np.argmin(distance[0])} 과 가장 가까울 것으로 예측됨")

In [ ]:
# KMeans 클래스의 가장 가까운 클러스터 중심을 예측 클래스로 출력하는 predict() 메소드를 활용

print(km.predict(fruits_2d[idx:idx+1]))

In [ ]:
# 해당 이미지를 출력함

draw_fruits(fruits[idx:idx+1])

### 데이터셋의 차원을 줄이면 학습 알고리즘의 속도를 크게 높일 수 있습니다.

---
### 데이터의 차원을 줄이면 시각화하기에 용이합니다.
### 3차원 이하로 줄이면 그래프로 나타낼 수 있습니다.


In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=0.5)
pca.fit(fruits_2d)

fruits_pca = pca.transform(fruits_2d)

for label in range(0, 3):
    data = fruits_pca[km.labels_ == label]
    plt.scatter(data[:,0], data[:,1])
plt.legend(['apple', 'banana', 'pineapple'])
plt.show()

### 사과와 파인애플 클러스터의 경계가 가깝게 붙어 있음.
### 두 클러스터 샘플은 몇 개가 혼동을 일으킬 수도 있음.
### 데이터 시각화를 통해 예상치 못한 통찰을 얻을 수 있음.